# IEX Cloud

https://algotrading101.com/learn/iex-api-guide/

Guide for getting data

In [ ]:
import os
import requests

base_url = 'https://workspace.iex.cloud/v1'
API_KEY = 'sk_b1349c1a99d7473aa22b6142a112ef3c'
token = os.environ.get(API_KEY)
params = {'token': token}



In [ ]:
resp = requests.get(base_url + '/status')

GET /stock/{symbol}/intraday-prices

In [ ]:
# endpoint = '/data/CORE/MINUTEBAR/AAPL/??from=2022-10-22??to=2022-10-23??token={}'.format(API_KEY)
# endpoint = '/data/CORE/MINUTEBAR/AAPL/??from=2022-10-22??to=2022-10-23??token={}'.format(API_KEY)
endpoint = '/stock/twtr/chart/{range}/{date}'
resp = requests.get(base_url+endpoint)

In [ ]:
resp.raise_for_status()

In [ ]:
resp

In [ ]:
import pandas as pd
df = pd.DataFrame(resp.json())

In [ ]:
resp.json()

In [ ]:
df.head()

# Polygon

https://polygon.io/dashboard

The aggregates endpoint has a max limit of 50,000. If querying 5 minute bars, it uses one minute as its base limit. This means that querying 5 minute bars with a max limit of 50k, will return 50k minutes worth of 5 minute bars. This is roughly 1-2 months. I recommend breaking up your aggregate requests into one to two month time frames.

In [19]:
import os
import time
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

API_KEY = '6BM1yLKk1jcJLKIpEvyKVc_jyCggOKij'


In [20]:
def shift_date(given_date='2020-10-25',n_months=1):
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(given_date, date_format)
    future_date = dtObj + relativedelta(months=n_months)
    # Convert datetime object to string in required format
    future_date_str = future_date.strftime(date_format)
    return future_date_str

def compare_dates(start,end,date_format='%Y-%m-%d'):
    start = datetime.strptime(start, date_format)
    end = datetime.strptime(end, date_format)
    return start>end

## get symbols

In [21]:
ndxt_df = pd.read_excel('data/custom_index.xlsx',index_col=0)
ndxt_df.columns = [n.split(' ')[0] for n in ndxt_df.columns]
ndxt_df.index = pd.to_datetime(ndxt_df.index)

In [22]:
symbols = ndxt_df.loc['2020-10-20':].dropna(axis=1).columns.to_list()

# check symbols

In [23]:
for s in symbols:
    r = requests.get('https://api.polygon.io/v3/reference/tickers?ticker={}&active=true&sort=ticker&order=asc&limit=10&apiKey=6BM1yLKk1jcJLKIpEvyKVc_jyCggOKij'.format(s))
    print(s)
    print(resp)

GOOGL
AMZN
RIOT
TSLA
SPY
AAPL
GME
BBBY
TLRY


In [17]:
symbols[0:]

['META', 'AMZN', 'RIOT', 'TSLA', 'SPY', 'AAPL', 'GME', 'BBBY', 'TLRY']

# get data

In [25]:
full_df_to_concat = []
for symbol in tqdm(symbols[0:]):
    df_to_concat = []
    start, end = '2020-10-25', '2022-10-19'
    print(symbol)
    while not compare_dates(start,end):
        startstep = shift_date(start,n_months=1)
        # print(start,startstep)
        URL = 'https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?adjusted=true&sort=asc&limit=50000&apiKey={}'.format(symbol,
                                                                                                                                start,
                                                                                                                                startstep,
                                                                                                                                API_KEY)

        resp = requests.get(URL)

        # decode data into pandas dataframe
        df = pd.DataFrame(resp.json()['results'])
        df.columns = ['volume','vwap','open','close','high','low','timestamp','transactions']
        df['ticker'] = resp.json()['ticker']
        df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')
        df_to_concat.append(df)
        start = startstep
        #time.sleep(25)

    cdf = pd.concat(df_to_concat) 
    cdf.to_csv('data/daily_data_test/{}.csv'.format(symbol))
    full_df_to_concat.append(cdf)
    
panel = pd.concat(full_df_to_concat)
# then save
panel.to_parquet('data/daily_data_test/custom_index.parquet.gzip',compression='gzip')

  0%|          | 0/9 [00:00<?, ?it/s]

GOOGL


 11%|█         | 1/9 [00:04<00:39,  4.98s/it]

AMZN


 22%|██▏       | 2/9 [00:10<00:35,  5.06s/it]

...

 78%|███████▊  | 7/9 [00:40<00:11,  5.71s/it]

BBBY


 89%|████████▉ | 8/9 [00:47<00:06,  6.06s/it]

TLRY


100%|██████████| 9/9 [00:52<00:00,  5.86s/it]


In [ ]:
resp.raise_for_status()

In [ ]:
cdf